<a href="https://colab.research.google.com/github/horsinnaround/Data-Science-Tools/blob/main/Tools_and_other_Stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [ ]:
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
import numpy as np
from sklearn.model_selection import cross_val_score
import zipfile
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from numpy import array
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB 
!pip install catboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier 
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import seaborn as sns
from time import time
from IPython.display import clear_output
clear_output()


# All the way with texts

## Text to int

for changing dtype of data frame columns to serial or factorize them or make strings ti ints

In [ ]:
# df_ = put dataframe here
str_to_fac=[]
for i in range(len(df.dtypes)):
    if df.dtypes[i] == 'object':
        to_fac = df[df.columns[i]].factorize()
        str_to_fac.append(to_fac)
        df_[f'{df.columns[i]}'] = to_fac[0]

## Text to vector

In [ ]:
# text to vector
# from sklearn.feature_extraction.text import TfidfVectorizer
# text is the column that you have the sentense
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.text).toarray()
labels = df.category_id
features.shape

# Regular ml

## classification with different data sample number

In [ ]:
%%time

# Learning curve:

scores_list = []

clf = CatBoostClassifier()
# clf = SVC()

def loss_func(Y_true, Y_prediction): # definig a loss function
    return (1 - accuracy_score(Y_true, Y_prediction))

ns_list = [1000,5000,9000,13000,19000,25000] # number of samples that we want to keep
n_avg = 1  # Number of times to average over

labels = list(df['category_id'])
for ns in ns_list:   # For different number of samples, we try training our estimator
    print(f"for {ns} number of data")        

    e_in = 0
    e_out = 0   # For each, we record the in and out score. 
    for i in range(n_avg):
        X_train, X_test, Y_train, Y_test = train_test_split(features[0:ns],labels[0:ns], random_state = 0)

        clf.fit( X_train , Y_train )

        e_in  += loss_func( Y_train, clf.predict(X_train ) ) # in-sample error
        e_out += loss_func( Y_test , clf.predict(X_test )  ) # out-sample error

    scores_list += [[e_in/n_avg, e_out/n_avg]]

    
scores_list = array(scores_list)  

xlabel = 'Training samples'
ylabel = 'loss'
plt_title = 'Learning curve for classification data \n using a CatBoost Classifier'
fig = plt.figure(figsize=(8,6),dpi=300)
ax2 = fig.add_subplot(111, xlabel=xlabel, ylabel=ylabel, title=plt_title)
ax2.plot(ns_list, scores_list[:,0], '-o',label = 'Training Score')
ax2.plot(ns_list, scores_list[:,1], '-o',label = 'Validation Score')
ax2.legend()
plt.savefig('learning curve.png')
plt.show()

## simply test

In [ ]:
%%time


# ns_list = [1000,5000,9000,13000,19000] # number of samples that we want to keep
n_avg = 3 # Number of times to average over
duration_fit_list=[]
duration_pred_list=[]
accuracy_score_list=[]
weighted_precision_score_list=[]
weighted_recall_score_list=[]
weighted_f1_score_list=[]
entries = []
clf = SVC()
duration_fit=0
duration_pred=0
accuracy_scores = 0
weighted_precision_scores = 0
weighted_recall_scores = 0
weighted_f1_scores = 0
# print("4")
featuresT,features_,labelsT,labels_ = train_test_split(features,labels)
e_in = 0
e_out = 0   # For each, we record the in and out score. 
for i in range(n_avg):
    
    start_fit = time()
    clf.fit(X = featuresT, y = labelsT)
    end_fit = time()
    duration_fit += end_fit - start_fit # time duration for fitting
    
    start_pred = time()
    y_prime = clf.predict(features_)
    end_pred = time()
    duration_pred += end_pred - start_pred # time duration for predicting
    
    accuracy_scores += round(accuracy_score(y_prime,labels_), 4)
    weighted_precision_scores += round(precision_score(y_prime,labels_, average='weighted',zero_division=1), 4)
    weighted_recall_scores += round(recall_score(y_prime,labels_, average='weighted'), 4)
    weighted_f1_scores += round(f1_score(y_prime,labels_, average='weighted'), 4)
duration_fit_list.append(duration_fit/n_avg)
duration_pred_list.append(duration_pred/n_avg)
accuracy_score_list.append(accuracy_scores/n_avg)
weighted_precision_score_list.append(weighted_precision_scores/n_avg)
weighted_recall_score_list.append(weighted_recall_scores/n_avg)
weighted_f1_score_list.append(weighted_f1_scores/n_avg)


report_data = pd.DataFrame(['SVC'], columns=['model'])
report_data['fit_duration'] = duration_fit_list
report_data['pred_duration'] = duration_pred_list
report_data['accuracy'] = accuracy_score_list
report_data['weighted_precision'] = weighted_precision_score_list
report_data['weighted_recall'] = weighted_recall_score_list
report_data['weighted_f1_score'] = weighted_f1_score_list

# report_data.to_excel(f'/content/drive/MyDrive/data/{folder}/Scores_for_all_models_smote.xlsx')

## cross validation

In [ ]:
model_list = [
    RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0, max_features=30),
    LinearSVC(),
    MultinomialNB(),
    SVC(),
    MLPClassifier(max_iter=500,early_stopping=True),
    XGBClassifier(),
    CatBoostClassifier(verbose=0),
    LogisticRegression(random_state=0,max_iter=1000),
]

CV = 3
cv_df = pd.DataFrame(index=range(CV * len(model_list)))
entries = []
for model in model_list:
  print(model)
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
# plt.figure(dpi=150)
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

# Data

In [ ]:
# importing required modules
from zipfile import ZipFile
import os
compression = zipfile.ZIP_DEFLATED
def get_all_file_paths(directory):

	# initializing empty file paths list
	file_paths = []

	# crawling through directory and subdirectories
	for root, directories, files in os.walk(directory):
		for filename in files:
			# join the two strings in order to form the full filepath.
			filepath = os.path.join(root, filename)
			file_paths.append(filepath)

	# returning all file paths
	return file_paths		

def main():
	# path to folder which needs to be zipped
	directory = '/content/results'

	# calling function to get all file paths in the directory
	file_paths = get_all_file_paths(directory)

	# printing the list of all files to be zipped
	print('Following files will be zipped:')
	for file_name in file_paths:
		print(file_name)

	# writing files to a zipfile
	with ZipFile('/content/drive/MyDrive/A Work/my_python_files.zip','w') as zip:
		# writing each file one by one
		for file in file_paths:
			zip.write(file,compress_type=compression)

	print('All files zipped successfully!')		


if __name__ == "__main__":
	main()


## remove data with very low sample

In [ ]:
features = df_p.drop('labels',axis=1)
labels = df_p['labels']
labels_t = Counter(labels)
print('Original dataset shape %s' % Counter(labels))
drop_dead = []
for lab in labels_t.keys():
    if len(df[df['labels']==lab])<300:
        drop_dead.append(lab)
        df.drop(index=df[df['labels']==lab].index,inplace=True)
print('Final dataset shape %s' % Counter(labels))

## svm smoth

In [ ]:
from imblearn.over_sampling import SVMSMOTE 
from imblearn.over_sampling import SVMSMOTE ,SMOTENC,ADASYN,KMeansSMOTE,BorderlineSMOTE
from imblearn.under_sampling import AllKNN,NearMiss
sm = SVMSMOTE(sampling_strategy='auto', 
              random_state=3, 
              k_neighbors=5, 
              n_jobs=None, 
              m_neighbors=10, 
              svm_estimator=None,
              out_step=0.5)
features, labels = sm.fit_resample(features, labels)

## change labels in dataframe

In [ ]:
index = df[df.labels!='normal'].index
df.loc[index] = 'nep'

## zip unzipping

In [ ]:
# Unzipping
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/A/hamed thesis/kdd_test.csv.zip', "r")
zip_ref.extractall('/content/drive/MyDrive/A/hamed thesis/')
zip_ref.close()

## PCA

In [ ]:
# from sklearn.decomposition import PCA
pca = PCA(n_components=100, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto',random_state=None)
features = pca.fit_transform(features)
features.shape

## barplot

In [ ]:
# Count Plot (a.k.a. Bar Plot)
plt.figure(figsize=(9,6),dpi=200)
sns.countplot(y='labels', data=df_)
# plt.xticks(rotation=-90)

## cor plot

In [ ]:
corr = df_.corr();
plt.figure(figsize=(9,9),dpi=100);
# sns.heatmap(corr,linewidths=.5);
sns.heatmap(corr);

plt.xticks(rotation=-90);
plt.yticks(rotation=0);

## Smothe

In [ ]:
from imblearn.over_sampling import SVMSMOTE 
print('Original dataset shape %s' % Counter(y))
sm = SVMSMOTE(sampling_strategy='auto', 
              random_state=1, 
              k_neighbors=5, 
              n_jobs=None, 
              m_neighbors=10, 
              svm_estimator=None,
              out_step=0.5)
X_t, y_t = sm.fit_resample(X_t, y_t)
print('Resampled dataset shape %s' % Counter(y_t))

## Test classification

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification

X, y = make_classification(n_classes=2, class_sep=2,
weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))
print('Resampled dataset shape %s' % Counter(y_t))

## kfold

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SVMSMOTE 
k_fold = KFold(n_splits=5,shuffle=True, random_state=2)

# sm = SVMSMOTE(sampling_strategy='auto', 
#               random_state=1, 
#               k_neighbors=5, 
#               n_jobs=None, 
#               m_neighbors=10, 
#               svm_estimator=None,
#               out_step=0.5)
# features, labels = sm.fit_resample(features, labels)
dataaa = []
p = 0
ll = 0
for train_indices, test_indices in k_fold.split(features, labels ):

    dataaa.append((p,(train_indices, test_indices)))

    print('Train: %s | test: %s' % (len(train_indices), len(test_indices)))
    X_t, y_t, X_, y_  = features.iloc[train_indices],labels.iloc[train_indices],features.iloc[test_indices],labels.iloc[test_indices]

    # clf = XGBClassifier()
    clf = LogisticRegression()
    clf.fit(X = X_t, y = y_t)
    y_pred = clf.predict(X_)
    print(p, metrics.accuracy_score(y_,y_pred))
    p = p + 1
    ll = ll + metrics.accuracy_score(y_,y_pred)
        # print(i,classification_report(y_,y_pred))

ll / (p)

## Cross vaidation Root

In [ ]:
model_list = [
    RandomForestClassifier(random_state= random_state),
    LinearSVC(),
    # # MultinomialNB(),
    DecisionTreeClassifier(random_state= random_state),
    SVC(),
    MLPClassifier(),
    XGBClassifier(random_state= random_state),
    # # CatBoostClassifier(verbose=0),
    LogisticRegression(random_state= random_state),
]


cv_df = pd.DataFrame(index=range(CV * len(model_list)))
entries = []
entriesOne = []
scors = ['accuracy', 'precision', 'recall', 'f1','roc_auc']
for model in model_list:
  print(model)
  model_name = model.__class__.__name__
  accuracies = cross_validate(model, features, labels, scoring=scors ,return_train_score=True, cv=k_fold)
  cl = list(accuracies.keys())

  enteryOne = [model_name]
  [enteryOne.append(accuracies[cl[i]].mean()) for i in range(len(cl))]
  entriesOne.append(enteryOne)


  for fold_idx in range(CV):
    entery = [model_name,fold_idx+1]
    [entery.append(accuracies[cl[i]][fold_idx]) for i in range(len(cl))]
    entries.append(entery)
clm =['model name','index fold']
clmOne =['model name']
[clm.append(i) for i in cl]
clm
[clmOne.append(i) for i in cl]
clmOne
cv_df = pd.DataFrame(entries, columns=clm)
cv_dfOne = pd.DataFrame(entriesOne, columns=clmOne)
clear_output()
# entries

In [ ]:
import sklearn
# sklearn.metrics.SCORERS.keys()
